<a href="https://colab.research.google.com/github/carloscesar182/ai_advanced_course/blob/main/Notebooks/MLAlgorithms/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # famosa biblioteca de manipulação de dados tabulários no python
import numpy as np # para tratar dados em forma de vetores e matrizes, muito usada para IA
import graphviz

# sklearn é amplamente utilizada na área de dados
from sklearn.model_selection import train_test_split # train_test_split é usada pra dividir os dados em conjunto de treino e teste
from sklearn.tree import DecisionTreeClassifier, export_graphviz # biblioteca pra criar nosso modelo da arvore de decisão e grafico da árvore de decisão
from sklearn.preprocessing import LabelEncoder # pra codificar dados categóricos em dados numéricos para que a biblioteca seja capaz de compreender e processar
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report # métricas

In [ ]:
# importando os dados
base = pd.read_csv("insurance.csv", keep_default_na=False) # parametro keep_default_na permite escolher se mantém ou não os valores nulos padrão da base
base.head()

In [ ]:
# dropar coluna unnamed
base = base.drop(columns=["Unnamed: 0"])
base.head()

In [ ]:
# definir as variáveis
y = base.iloc[:,7].values # variável dependente
X = base.drop(base.columns[7], axis=1).values # variável independente
X

In [ ]:
# transformar variáveis categóricas em numéricas
# laço pra detectar apenas os objetos que são categóricos, que no pandas é o tipo object e transformar só eles
labelencoder = LabelEncoder()
for i in range(X.shape[1]): # percorre todas as colunas
  if X[:,i].dtype == 'object': # verifica se as colunas do laço são do tipo object
    X[:,i] = labelencoder.fit_transform(X[:,i]) # se forem, pega o labelencoder da coluna atual, transforma e coloca no mesmo objeto da coluna atual. Isso vai substituindo as colunas pela mesma informação codificada

In [ ]:
# dividir os dados em treino e teste
# precisamos ter 4 variáveis pra treino e teste. Independente de treino e teste e dependente de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12) # o test_size = 0.3 indica que 30% das linhas do CSV serão pra teste. Random_state garante que toda vez que rode, tenha o mesmo resultado

In [ ]:
# criar e treinar o modelo decisiontree
# cria o objeto limitando a complexidade da arvore com 8 nós de profundidade e 6 folhas pra ajudar minimizar o overfitting
# isso não significa que são os melhores parametros, pois o certo é aplicar os calculos antes de definir o tamanho
modelo = DecisionTreeClassifier(random_state=1, max_depth=8, max_leaf_nodes=6)
modelo.fit(X_train, y_train) # treina o modelo

In [ ]:
# criar a árvore graficamente
dot_data = export_graphviz(modelo, out_file=None, feature_names=base.columns[:-1],
                           class_names=True, rounded=True, filled=True)
graph = graphviz.Source(dot_data)
graph.render("decision_tree", format="png")
graph.view()

'decision_tree.pdf'

In [ ]:
previsoes = modelo.predict(X_test)
previsoes

array(['Moderate', 'Mild', 'Severe', ..., 'None', 'Mild', 'None'],
      dtype=object)

In [ ]:
# metricas
accuracy = accuracy_score(y_test, previsoes) # calcula o percentual de acerto comparando os dados do que aconteceu com o que foi previsto
precision = precision_score(y_test, previsoes, average='weighted') # vai considerar as classes de forma ponderada
recall = recall_score(y_test, previsoes, average='weighted')
f1 = f1_score(y_test, previsoes, average='weighted')
print(f'Acurácia: {accuracy:.2f}')
print(f'Precisão: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')

Acurácia: 0.94
Precisão: 0.93
Recall: 0.94
F1-score: 0.93


In [ ]:
# gerar o classification report
report = classification_report(y_test, previsoes)
print(report)

              precision    recall  f1-score   support

        Mild       0.87      0.71      0.78       570
    Moderate       0.75      0.68      0.71       488
        None       0.98      1.00      0.99      4253
      Severe       0.87      0.93      0.90       689

    accuracy                           0.94      6000
   macro avg       0.87      0.83      0.84      6000
weighted avg       0.93      0.94      0.93      6000

